In [1]:
# ===== ProDE-BT – Cell P1 (Eclipse): data, vocab, splits, loaders =====

import json
from collections import Counter

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------
# 1) Load Eclipse JSON files
# -----------------------------
DATA_ROOT    = "/content/drive/MyDrive/msr2013-bug_dataset-master/data/v02"
ECLIPSE_DIR  = f"{DATA_ROOT}/eclipse"

ASSIGNED_FILE   = f"{ECLIPSE_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{ECLIPSE_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{ECLIPSE_DIR}/bug_status.json"
RESOLUTION_FILE = f"{ECLIPSE_DIR}/resolution.json"

print("Eclipse folder:", ECLIPSE_DIR)

def load_last_event_table(path, root_key, out_col):
    """
    MSR2013 JSON format:
      { root_key : { bug_id : [ {when, what, who}, ... ], ... } }

    For each bug_id we keep the LAST event (max 'when') and its 'what'.
    """
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)
    inner = raw[root_key]

    rows = []
    for bug_id, events in inner.items():
        if not events:
            continue
        last = max(events, key=lambda e: e.get("when", 0))
        rows.append({
            "bug_id": int(bug_id),
            out_col: str(last.get("what", ""))
        })
    return pd.DataFrame(rows)

assigned_df   = load_last_event_table(ASSIGNED_FILE,   "assigned_to", "assigned_to")
short_df      = load_last_event_table(SHORT_DESC_FILE, "short_desc",  "short_desc")
bug_status_df = load_last_event_table(BUG_STATUS_FILE, "bug_status",  "bug_status")
resolution_df = load_last_event_table(RESOLUTION_FILE, "resolution",  "resolution")

# Merge into one table
df = (assigned_df
      .merge(short_df,      on="bug_id", how="inner")
      .merge(bug_status_df, on="bug_id", how="inner")
      .merge(resolution_df, on="bug_id", how="inner"))

print("Merged rows before filters:", len(df))

# -----------------------------
# 2) Filter to RESOLVED/VERIFIED/CLOSED AND FIXED
# -----------------------------
status_upper = df["bug_status"].str.upper()
res_upper    = df["resolution"].str.upper()

mask_status = status_upper.isin(["RESOLVED", "VERIFIED", "CLOSED"])
mask_res    = res_upper.str.contains("FIXED", na=False)

df = df[mask_status & mask_res].copy()
print("Rows after resolved+fixed filter:", len(df))

# -----------------------------
# 3) Keep only developers with >= MIN_BUGS bugs
# -----------------------------
MIN_BUGS = 2  # you can bump this to 5 or 10 if you want

df["owner"]       = df["assigned_to"].astype(str)
df["issue_title"] = df["short_desc"].astype(str)
df["description"] = ""   # no long description here

counts_per_owner = df["owner"].value_counts()
keep_owners = counts_per_owner[counts_per_owner >= MIN_BUGS].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after MIN_BUGS filter:", len(df))
print(f"Number of developers with ≥{MIN_BUGS} bugs:",
      df["owner"].nunique())

# Encode dev_id
df["dev_id"] = df["owner"].astype("category").cat.codes
N_CLASSES = df["dev_id"].nunique()
print("Final number of developers:", N_CLASSES)

# Classifier-style view
clf_df = df[["owner", "issue_title", "description", "dev_id"]]

# -----------------------------
# 4) Tokenisation (NLTK) + vocab
# -----------------------------
nltk.download("punkt", quiet=True)
try:
    nltk.data.find("tokenizers/punkt_tab/english.pickle")
except LookupError:
    nltk.download("punkt_tab", quiet=True)

def tokenize(text):
    return [t.lower() for t in word_tokenize(str(text))]

print("Tokenising all Eclipse bug reports...")
clf_df["text"] = (clf_df["issue_title"].fillna("").astype(str) +
                  " " +
                  clf_df["description"].fillna("").astype(str))
clf_df["tokens"] = clf_df["text"].apply(tokenize)

MAX_VOCAB = 50000
PAD_TOKEN = "<PAD>"

freq = Counter()
for toks in clf_df["tokens"]:
    freq.update(toks)

vocab = {PAD_TOKEN: 0}
for w, _ in freq.most_common(MAX_VOCAB - 1):
    if w not in vocab:
        vocab[w] = len(vocab)

VOCAB_SIZE = len(vocab)
print("Vocab size after cutoff:", VOCAB_SIZE)

# -----------------------------
# 5) Tokens → ids, padding
# -----------------------------
MAX_LEN = 300

def tokens_to_ids(tokens):
    ids = [vocab.get(t, 0) for t in tokens]   # OOV -> 0
    if len(ids) > MAX_LEN:
        ids = ids[:MAX_LEN]
    elif len(ids) < MAX_LEN:
        ids += [0] * (MAX_LEN - len(ids))
    return ids

clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)

X_all = np.vstack(clf_df["token_ids"].values).astype("int64")
y_all = clf_df["dev_id"].to_numpy(dtype=np.int64)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

# -----------------------------
# 6) Train / val / test split: 80 / 10 / 10
#    - FIRST split stratified
#    - SECOND split RANDOM (no stratify) to avoid single-class error
# -----------------------------
indices = np.arange(len(y_all))

# 80% train, 20% temp (val+test), stratified
train_idx, temp_idx, y_train, y_temp = train_test_split(
    indices,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# Split temp into 10% val, 10% test (no stratify here)
val_idx, test_idx, y_val, y_test = train_test_split(
    temp_idx,
    y_temp,
    test_size=0.5,         # 50/50 of the 20% → 10% + 10%
    random_state=42,
    stratify=None,         # IMPORTANT: no stratify to avoid 1-sample classes
)

print(f"Train bugs: {len(train_idx)}  Val bugs: {len(val_idx)}  Test bugs: {len(test_idx)}")

# -----------------------------
# 7) Dataset + DataLoaders
# -----------------------------
class BugDatasetWithIndex(Dataset):
    """
    Returns (tokens, label, global_index) for ProDE-BT.
    """
    def __init__(self, X, y, indices):
        self.X = X
        self.y = y
        self.indices = np.array(indices, dtype=np.int64)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        idx = int(self.indices[i])
        x  = torch.from_numpy(self.X[idx])
        y  = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y, idx

BATCH_SIZE = 256

train_ds = BugDatasetWithIndex(X_all, y_all, train_idx)
val_ds   = BugDatasetWithIndex(X_all, y_all, val_idx)
test_ds  = BugDatasetWithIndex(X_all, y_all, test_idx)
all_ds   = BugDatasetWithIndex(X_all, y_all, np.arange(len(y_all)))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  drop_last=False)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
all_loader   = DataLoader(all_ds,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print("DataLoaders ready.")

xb, yb, idxb = next(iter(train_loader))
print("Example batch shapes -> xb:", xb.shape, " yb:", yb.shape, " idx:", idxb.shape)


Mounted at /content/drive
Using device: cuda
Eclipse folder: /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse
Merged rows before filters: 165547
Rows after resolved+fixed filter: 76268
Rows after MIN_BUGS filter: 76185
Number of developers with ≥2 bugs: 439
Final number of developers: 439
Tokenising all Eclipse bug reports...


/tmp/ipython-input-249353971.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["text"] = (clf_df["issue_title"].fillna("").astype(str) +
/tmp/ipython-input-249353971.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["tokens"] = clf_df["text"].apply(tokenize)


Vocab size after cutoff: 33390


/tmp/ipython-input-249353971.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)


X_all shape: (76185, 300)
y_all shape: (76185,)
Train bugs: 60948  Val bugs: 7618  Test bugs: 7619
DataLoaders ready.
Example batch shapes -> xb: torch.Size([256, 300])  yb: torch.Size([256])  idx: torch.Size([256])


In [2]:
# ===== ProDE-BT on Eclipse – Cell P2: encoder + model =====

import torch.nn as nn
import torch.nn.functional as F

EMBED_DIM  = 200      # richer encoder for ProDE-BT
HIDDEN_DIM = 256

class BugEncoder(nn.Module):
    """
    Text encoder: Embedding -> BiLSTM -> attention pooling -> tanh projection.
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm      = nn.LSTM(
            embed_dim,
            hidden_dim,
            batch_first=True,
            bidirectional=True,
            num_layers=2,
            dropout=dropout,
        )
        self.attn      = nn.Linear(2 * hidden_dim, 1)
        self.proj      = nn.Linear(2 * hidden_dim, hidden_dim)
        self.dropout   = nn.Dropout(dropout)

    def forward(self, input_ids):
        # input_ids: [B, T]
        mask = (input_ids != 0).unsqueeze(-1)          # [B, T, 1]

        x = self.embedding(input_ids)                  # [B, T, E]
        x, _ = self.lstm(x)                            # [B, T, 2H]

        scores = torch.tanh(self.attn(x)).squeeze(-1)  # [B, T]
        scores = scores.masked_fill(mask.squeeze(-1) == 0, -1e9)
        alpha  = torch.softmax(scores, dim=1).unsqueeze(-1)   # [B, T, 1]

        pooled = (alpha * x).sum(dim=1)                # [B, 2H]
        pooled = self.dropout(pooled)
        z = torch.tanh(self.proj(pooled))              # [B, H]
        return z


class ProDEBTModel(nn.Module):
    """
    ProDE-BT: bug encoder + classifier.
    Stage-1: hard labels; Stage-2: profile-driven soft labels.
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim, n_classes):
        super().__init__()
        self.encoder = BugEncoder(vocab_size, embed_dim, hidden_dim)
        self.fc      = nn.Linear(hidden_dim, n_classes)

    def forward(self, input_ids):
        z = self.encoder(input_ids)        # [B, H]
        logits = self.fc(z)                # [B, C]
        return logits, z

VOCAB_SIZE = int(VOCAB_SIZE)
model = ProDEBTModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, N_CLASSES).to(device)
print(model)


ProDEBTModel(
  (encoder): BugEncoder(
    (embedding): Embedding(33390, 200, padding_idx=0)
    (lstm): LSTM(200, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (attn): Linear(in_features=512, out_features=1, bias=True)
    (proj): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (fc): Linear(in_features=256, out_features=439, bias=True)
)


In [3]:
# ===== ProDE-BT on Eclipse – Cell P3: Stage-1 training (hard labels) =====

EPOCHS_STAGE1 = 8
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

best_val_loss_stage1 = float("inf")
best_state_stage1 = None

def run_epoch_ce(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()
    total_loss = 0.0
    total_items = 0
    with torch.set_grad_enabled(train):
        for xb, yb, _ in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            if train:
                optimizer.zero_grad()
            logits, _ = model(xb)
            loss = criterion(logits, yb)
            if train:
                loss.backward()
                optimizer.step()
            total_loss += loss.item() * xb.size(0)
            total_items += xb.size(0)
    return total_loss / max(total_items, 1)

for epoch in range(1, EPOCHS_STAGE1 + 1):
    train_loss = run_epoch_ce(train_loader, train=True)
    val_loss   = run_epoch_ce(val_loader,   train=False)
    print(f"Epoch {epoch:02d}  train_loss={train_loss:.4f}  val_loss={val_loss:.4f}")

    if val_loss < best_val_loss_stage1 - 1e-4:
        best_val_loss_stage1 = val_loss
        best_state_stage1 = {k: v.cpu().clone() for k, v in model.state_dict().items()}

print("Stage-1 training finished. Best val loss:", best_val_loss_stage1)

if best_state_stage1 is not None:
    model.load_state_dict(best_state_stage1)
    model.to(device)


Epoch 01  train_loss=4.1932  val_loss=3.7562
Epoch 02  train_loss=3.4397  val_loss=3.2441
Epoch 03  train_loss=3.0153  val_loss=3.0898
Epoch 04  train_loss=2.7319  val_loss=3.0421
Epoch 05  train_loss=2.4714  val_loss=3.0781
Epoch 06  train_loss=2.1986  val_loss=3.1836
Epoch 07  train_loss=1.9161  val_loss=3.3767
Epoch 08  train_loss=1.6635  val_loss=3.6050
Stage-1 training finished. Best val loss: 3.042126305180292


In [4]:
# ===== ProDE-BT on Eclipse – Cell P4: bug embeddings + developer profiles =====

import numpy as np

N_BUGS   = X_all.shape[0]
y_all_np = y_all.astype(np.int64)
n_devs   = N_CLASSES

model.eval()
bug_repr_all = np.zeros((N_BUGS, HIDDEN_DIM), dtype=np.float32)

with torch.no_grad():
    for xb, yb, idx in all_loader:
        xb  = xb.to(device, non_blocking=True)
        _, z = model(xb)                  # [B, H]
        idx_np = idx.cpu().numpy()
        bug_repr_all[idx_np] = z.cpu().numpy()

print("Bug representation matrix:", bug_repr_all.shape)

# Aggregate to developer profiles (mean of training bugs)
dev_repr   = np.zeros((n_devs, HIDDEN_DIM), dtype=np.float32)
dev_counts = np.zeros(n_devs, dtype=np.int64)

for g_idx in train_idx:
    d = int(y_all_np[g_idx])
    dev_repr[d] += bug_repr_all[g_idx]
    dev_counts[d] += 1

nonzero = dev_counts > 0
dev_repr[nonzero] /= dev_counts[nonzero][:, None]

print("Developer profile matrix:", dev_repr.shape)
print("Devs with no training bugs:", np.sum(~nonzero))

def l2_normalise(arr, axis=-1, eps=1e-8):
    norm = np.linalg.norm(arr, axis=axis, keepdims=True)
    return arr / (norm + eps)

bug_repr_all_norm = l2_normalise(bug_repr_all)
dev_repr_norm     = l2_normalise(dev_repr)

DEV_PROF_TORCH = torch.tensor(dev_repr_norm, dtype=torch.float32, device=device)
print("DEV_PROF_TORCH:", DEV_PROF_TORCH.shape)


Bug representation matrix: (76185, 256)
Developer profile matrix: (439, 256)
Devs with no training bugs: 0
DEV_PROF_TORCH: torch.Size([439, 256])


In [5]:
# ===== ProDE-BT on Eclipse – Cell P5: soft-label generator (Stage-2) =====

TOP_M     = 5     # top profile neighbours
ALPHA     = 0.10   # mix weight for profile-based distribution
TEMP_SOFT = 0.5    # temperature for top-M softmax

def make_soft_labels(global_indices):
    """
    global_indices: LongTensor or numpy array of bug indices in this batch.
    Returns:
        soft_targets: [B, C]
        true_devs   : [B]
    """
    if isinstance(global_indices, torch.Tensor):
        idx_np = global_indices.cpu().numpy()
    else:
        idx_np = np.asarray(global_indices, dtype=np.int64)

    B = len(idx_np)
    C = n_devs

    bug_vec = bug_repr_all_norm[idx_np]   # [B, H]
    bug_vec_t = torch.tensor(bug_vec, dtype=torch.float32, device=device)

    sim = torch.matmul(bug_vec_t, DEV_PROF_TORCH.t())  # [B, C]

    M = min(TOP_M, C)
    top_vals, top_idx = torch.topk(sim, k=M, dim=1)    # [B, M]

    prob_top = torch.softmax(top_vals / TEMP_SOFT, dim=1)  # [B, M]

    soft = torch.zeros(B, C, device=device)
    soft.scatter_(1, top_idx, prob_top)

    true_devs = torch.tensor(y_all_np[idx_np], dtype=torch.long, device=device)
    one_hot = torch.zeros_like(soft)
    one_hot.scatter_(1, true_devs.view(-1, 1), 1.0)

    soft = (1.0 - ALPHA) * one_hot + ALPHA * soft
    soft = soft / soft.sum(dim=1, keepdim=True)

    return soft, true_devs


In [6]:
# ===== ProDE-BT on Eclipse – Cell P6: Stage-2 training (hard + soft labels) =====

EPOCHS_STAGE2 = 6
BETA = 0.05      # weight for soft-label loss

best_val_loss_stage2 = float("inf")
best_state_stage2 = None

for epoch in range(1, EPOCHS_STAGE2 + 1):
    model.train()
    total_loss = 0.0
    total_items = 0

    for xb, yb, idx_global in train_loader:
        xb = xb.to(device, non_blocking=True)
        idx_global = idx_global.to(device, non_blocking=True)

        soft_targets, true_devs = make_soft_labels(idx_global)

        optimizer.zero_grad()
        logits, _ = model(xb)
        log_probs = F.log_softmax(logits, dim=1)

        loss_hard = F.nll_loss(log_probs, true_devs)
        loss_soft = -(soft_targets * log_probs).sum(dim=1).mean()
        loss = (1.0 - BETA) * loss_hard + BETA * loss_soft

        loss.backward()
        optimizer.step()

        total_loss   += loss.item() * xb.size(0)
        total_items  += xb.size(0)

    val_loss = run_epoch_ce(val_loader, train=False)
    print(f"[Stage-2] Epoch {epoch:02d}  train_loss={total_loss/total_items:.4f}  val_ce={val_loss:.4f}")

    if val_loss < best_val_loss_stage2 - 1e-4:
        best_val_loss_stage2 = val_loss
        best_state_stage2 = {k: v.cpu().clone() for k, v in model.state_dict().items()}

print("Stage-2 ProDE-BT training finished. Best val CE:", best_val_loss_stage2)

if best_state_stage2 is not None:
    model.load_state_dict(best_state_stage2)
    model.to(device)


[Stage-2] Epoch 01  train_loss=2.4159  val_ce=3.1013
[Stage-2] Epoch 02  train_loss=2.2243  val_ce=3.1762
[Stage-2] Epoch 03  train_loss=2.0197  val_ce=3.3059
[Stage-2] Epoch 04  train_loss=1.8174  val_ce=3.4890
[Stage-2] Epoch 05  train_loss=1.5898  val_ce=3.7046
[Stage-2] Epoch 06  train_loss=1.3847  val_ce=3.8954
Stage-2 ProDE-BT training finished. Best val CE: 3.1013415950150214


In [7]:
# ===== ProDE-BT on Eclipse – Cell P7: temperature scaling + calibration metrics =====

model.eval()

val_logits_list = []
val_labels_list = []

with torch.no_grad():
    for xb, yb, _ in val_loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        logits, _ = model(xb)
        val_logits_list.append(logits)
        val_labels_list.append(yb)

val_logits = torch.cat(val_logits_list, dim=0)
val_labels = torch.cat(val_labels_list, dim=0)
print("Validation logits:", val_logits.shape)

# --- optimise temperature ---
log_T = torch.zeros(1, device=device, requires_grad=True)
optimizer_T = torch.optim.LBFGS([log_T], lr=0.1, max_iter=50)

def eval_T():
    optimizer_T.zero_grad()
    T = torch.exp(log_T)
    scaled = val_logits / T
    loss = F.cross_entropy(scaled, val_labels)
    loss.backward()
    return loss

optimizer_T.step(eval_T)
T_opt = torch.exp(log_T).detach()
print("Optimal temperature T:", float(T_opt))

# --- ECE, Brier, Coverage@0.9 ---
import numpy as np

def compute_ece_brier(logits, labels, T, n_bins=15, target_coverage=0.9):
    probs_before = F.softmax(logits, dim=1).cpu().numpy()
    probs_after  = F.softmax(logits / T, dim=1).cpu().numpy()
    y_true       = labels.cpu().numpy()
    N, C         = probs_before.shape

    def ece(probs, y_true, n_bins=15):
        confidences = probs.max(axis=1)
        predictions = probs.argmax(axis=1)
        accuracies  = (predictions == y_true)

        bins = np.linspace(0.0, 1.0, n_bins + 1)
        ece_val = 0.0
        for i in range(n_bins):
            lo, hi = bins[i], bins[i+1]
            mask = (confidences > lo) & (confidences <= hi)
            if not np.any(mask):
                continue
            conf_bin = confidences[mask].mean()
            acc_bin  = accuracies[mask].mean()
            ece_val += np.abs(acc_bin - conf_bin) * (mask.sum() / len(y_true))
        return ece_val

    def brier(probs, y_true, n_classes):
        one_hot = np.eye(n_classes)[y_true]
        return np.mean((probs - one_hot) ** 2)

    ece_before = ece(probs_before, y_true, n_bins)
    ece_after  = ece(probs_after,  y_true, n_bins)
    b_before   = brier(probs_before, y_true, C)
    b_after    = brier(probs_after,  y_true, C)

    def coverage_and_size(probs, y_true, target=0.9):
        cover_flags = []
        sizes = []
        for i in range(len(y_true)):
            p = probs[i]
            order = np.argsort(-p)
            cum = 0.0
            k = 0
            for j in order:
                cum += p[j]
                k += 1
                if cum >= target:
                    break
            sizes.append(k)
            cover_flags.append(int(y_true[i] in order[:k]))
        return float(np.mean(cover_flags)), float(np.mean(sizes))

    cov_before, size_before = coverage_and_size(probs_before, y_true, target_coverage)
    cov_after,  size_after  = coverage_and_size(probs_after,  y_true, target_coverage)

    return (ece_before, ece_after,
            b_before, b_after,
            cov_before, cov_after,
            size_before, size_after)

ece_b, ece_a, b_b, b_a, cov_b, cov_a, size_b, size_a = compute_ece_brier(
    val_logits, val_labels, T_opt, n_bins=15, target_coverage=0.9
)

print(f"ECE  before: {ece_b:.4f}   after: {ece_a:.4f}")
print(f"Brier before: {b_b:.4f}   after: {b_a:.4f}")
print(f"Coverage@0.9 before: {cov_b:.3f} (avg set size {size_b:.2f})")
print(f"Coverage@0.9 after : {cov_a:.3f} (avg set size {size_a:.2f})")


Validation logits: torch.Size([7618, 439])
Optimal temperature T: 1.1736063957214355
ECE  before: 0.0926   after: 0.0283
Brier before: 0.0020   after: 0.0019
Coverage@0.9 before: 0.822 (avg set size 25.31)
Coverage@0.9 after : 0.901 (avg set size 45.90)


In [8]:
# ===== ProDE-BT on Eclipse – Cell P8: profile-augmented predictions + Hit@K =====

GAMMA_PROFILE = 2.0   # scale for profile-based logits (tune 2–10 if needed)

def predict_probs(loader):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for xb, yb, _ in loader:
            xb = xb.to(device, non_blocking=True)

            logits_base, z = model(xb)                 # [B, C], [B, H]

            # profile scores from dev profiles
            z_norm   = F.normalize(z, p=2, dim=1)
            dev_norm = F.normalize(DEV_PROF_TORCH, p=2, dim=1)
            profile_scores = torch.matmul(z_norm, dev_norm.t())  # [B, C]

            logits = logits_base + GAMMA_PROFILE * profile_scores
            scaled = logits / T_opt
            probs  = F.softmax(scaled, dim=1)

            all_probs.append(probs.cpu().numpy())
            all_labels.append(yb.numpy())

    return np.vstack(all_probs), np.concatenate(all_labels)

probs_test, y_test_np = predict_probs(test_loader)

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

TOPKS = [1, 3, 5, 7, 9]
hrs_prode = []

print("=== ProDE-BT on Eclipse (profile-driven, noise-robust, calibrated) ===")
for k in TOPKS:
    hr = hit_at_k(probs_test, y_test_np, k)
    hrs_prode.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr_prode = float(np.mean(hrs_prode))
print(f"Average Hit Ratio ProDE-BT (Eclipse) : {avg_hr_prode:.4f}")


=== ProDE-BT on Eclipse (profile-driven, noise-robust, calibrated) ===
Top-1 Hit Ratio: 0.2947
Top-3 Hit Ratio: 0.5186
Top-5 Hit Ratio: 0.6228
Top-7 Hit Ratio: 0.6850
Top-9 Hit Ratio: 0.7241
Average Hit Ratio ProDE-BT (Eclipse) : 0.5690
